# Instruction Power Differences

Supported setups:

SCOPES:

* OPENADC
* CWNANO

PLATFORMS:

* CWLITEARM
* CWLITEXMEGA
* CWNANO

This tutorial will introduce you to measuring the power consumption of a device under attack. It will demonstrate how the power consumption of a target changes based on what operations it's doing.

If you haven't yet, you should probably complete Tutorial B1, which introduces building firmware, programming the target, and scripting.

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'

## Setting up Firmware

In this tutorial, we will once again be working off of the simpleserial-base firmware.

Let's start by creating a new project and building our firmware:

In [2]:
%%bash
cd ../hardware/victims/firmware/
mkdir -p simpleserial-base-lab2 && cp -r simpleserial-base/* $_
cd simpleserial-base-lab2

In [3]:
CRYPTO_TARGET = "NONE"

In [4]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=$2

SS_VER set to SS_VER_1_1
rm -f -- simpleserial-base-CWLITEARM.hex
rm -f -- simpleserial-base-CWLITEARM.eep
rm -f -- simpleserial-base-CWLITEARM.cof
rm -f -- simpleserial-base-CWLITEARM.elf
rm -f -- simpleserial-base-CWLITEARM.map
rm -f -- simpleserial-base-CWLITEARM.sym
rm -f -- simpleserial-base-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-base.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- simpleserial-base.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- simpleserial-base.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:4.9.3+svn231177-1) 4.9.3 20150529 (prerelease)
Copyright (C) 2014 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: simples

/usr/lib/gcc/arm-none-eabi/4.9.3/../../../arm-none-eabi/bin/ld: warning: /usr/lib/gcc/arm-none-eabi/4.9.3/../../../arm-none-eabi/lib/armv7e-m/libg_nano.a(lib_a-atexit.o) uses 2-byte wchar_t yet the output is to use 4-byte wchar_t; use of wchar_t values across objects may fail
/usr/lib/gcc/arm-none-eabi/4.9.3/../../../arm-none-eabi/bin/ld: warning: /usr/lib/gcc/arm-none-eabi/4.9.3/../../../arm-none-eabi/lib/armv7e-m/libg_nano.a(lib_a-errno.o) uses 2-byte wchar_t yet the output is to use 4-byte wchar_t; use of wchar_t values across objects may fail
/usr/lib/gcc/arm-none-eabi/4.9.3/../../../arm-none-eabi/bin/ld: warning: /usr/lib/gcc/arm-none-eabi/4.9.3/../../../arm-none-eabi/lib/armv7e-m/libg_nano.a(lib_a-exit.o) uses 2-byte wchar_t yet the output is to use 4-byte wchar_t; use of wchar_t values across objects may fail
/usr/lib/gcc/arm-none-eabi/4.9.3/../../../arm-none-eabi/bin/ld: warning: /usr/lib/gcc/arm-none-eabi/4.9.3/../../../arm-none-eabi/lib/armv7e-m/libg_nano.a(lib_a-fini.o) uses

As in the previous tutorial, we'll need to modify our firmware. Navigate to the `get_pt()` function:
```C
uint8_t get_pt(uint8_t* pt)
{
	/**********************************
	* Start user-specific code here. */
	trigger_high();
	
	//16 hex bytes held in 'pt' were sent
	//from the computer. Store your response
	//back into 'pt', which will send 16 bytes
	//back to computer. Can ignore of course if
	//not needed
	
	trigger_low();
	/* End user-specific code here. *
	********************************/
	simpleserial_put('r', 16, pt);
	return 0x00;
}

```

To start off, we'll add a simple `for` loop. We'll start off by looking at how the power trace changes based on how long the loop is. Start with a loop (make sure your variables are volatile) that runs from 0 to 4:

```C
for(volatile int i = 0; i < 5; i++);
```

Next, we'll move on to actually capturing and displaying the power trace.

## ChipWhisperer Setup

Setup for this tutorial will be pretty similar to Tutorial B1, so we'll skip most of it by calling some helper scripts. This setup should work for most targets, but if you're using a target other than the XMEGA or STM32F3 (CWLite w/ Arm), you may need to call a different script or do additional setup (like programming the target with an external programmer). See the wiki page for your target for more information.

If you're curious about what's happening in these helper scripts, they're typically located in the `Helper_Scripts` folder.

In [5]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=$2

SS_VER set to SS_VER_1_1
rm -f -- simpleserial-base-CWLITEARM.hex
rm -f -- simpleserial-base-CWLITEARM.eep
rm -f -- simpleserial-base-CWLITEARM.cof
rm -f -- simpleserial-base-CWLITEARM.elf
rm -f -- simpleserial-base-CWLITEARM.map
rm -f -- simpleserial-base-CWLITEARM.sym
rm -f -- simpleserial-base-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-base.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- simpleserial-base.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- simpleserial-base.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:4.9.3+svn231177-1) 4.9.3 20150529 (prerelease)
Copyright (C) 2014 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: simples

/usr/lib/gcc/arm-none-eabi/4.9.3/../../../arm-none-eabi/bin/ld: warning: /usr/lib/gcc/arm-none-eabi/4.9.3/../../../arm-none-eabi/lib/armv7e-m/libg_nano.a(lib_a-atexit.o) uses 2-byte wchar_t yet the output is to use 4-byte wchar_t; use of wchar_t values across objects may fail
/usr/lib/gcc/arm-none-eabi/4.9.3/../../../arm-none-eabi/bin/ld: warning: /usr/lib/gcc/arm-none-eabi/4.9.3/../../../arm-none-eabi/lib/armv7e-m/libg_nano.a(lib_a-errno.o) uses 2-byte wchar_t yet the output is to use 4-byte wchar_t; use of wchar_t values across objects may fail
/usr/lib/gcc/arm-none-eabi/4.9.3/../../../arm-none-eabi/bin/ld: warning: /usr/lib/gcc/arm-none-eabi/4.9.3/../../../arm-none-eabi/lib/armv7e-m/libg_nano.a(lib_a-exit.o) uses 2-byte wchar_t yet the output is to use 4-byte wchar_t; use of wchar_t values across objects may fail
/usr/lib/gcc/arm-none-eabi/4.9.3/../../../arm-none-eabi/bin/ld: warning: /usr/lib/gcc/arm-none-eabi/4.9.3/../../../arm-none-eabi/lib/armv7e-m/libg_nano.a(lib_a-fini.o) uses

In [6]:
%run "Helper_Scripts/Setup_Generic.ipynb"

Traceback (most recent call last):
  File "<ipython-input-6-4b322b8b6364>", line 3, in <module>
    if not scope.connectStatus:
NameError: name 'scope' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ms/chipwhisperer/software/chipwhisperer/hardware/naeusb/naeusb.py", line 295, in txrx
    response = self.open(serial_number=payload)
  File "/home/ms/chipwhisperer/software/chipwhisperer/hardware/naeusb/naeusb.py", line 345, in open
    dev.set_configuration(0)
  File "/home/ms/.local/lib/python3.7/site-packages/usb/core.py", line 869, in set_configuration
    self._ctx.managed_set_configuration(self, configuration)
  File "/home/ms/.local/lib/python3.7/site-packages/usb/core.py", line 102, in wrapper
    return f(self, *args, **kwargs)
  File "/home/ms/.local/lib/python3.7/site-packages/usb/core.py", line 148, in managed_set_configuration
    self.backend.set_configuration(self.handle, cfg.bConfiguratio

Warning: Could not connect to "NewAE USB (CWLite/CW1200)". It may have been disconnected, is in an error state, or is being used by another tool.

By default, the scope will capture many more traces than we need, so we'll reduce that to 1000.

In [7]:
scope.adc.samples = 1000
fw_path = '../hardware/victims/firmware/simpleserial-base-lab2/simpleserial-base-{}.hex'.format(PLATFORM)

NameError: name 'scope' is not defined

Programming is the same as in the last part:

In [ ]:
cw.program_target(scope, prog, fw_path)

## Capturing Traces

In [ ]:
print(scope)

Like before, most of this should look familiar from the last tutorial. We'll start by programming the target, then capturing a trace, and finally displaying it using bokeh. We don't really care about what the target responds with this time, so we won't do anything with what we read back.

In [ ]:
import holoviews as hv
hv.extension('bokeh')
import numpy as np
ktp = cw.ktp.Basic()
key, text = ktp.next()  # manual creation of a key, text pair can be substituted here

trace = cw.capture_trace(scope, target, text)
hv.Curve(trace.wave).opts(width=600, height=600)

## Further Modifications

You might be able to tell where our loop is happening, but to make sure, let's modify it and see how the trace changes. Change when the loop ends and save the file. Rebuild the file:

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=$2

Reprogram the target:

In [ ]:
cw.program_target(scope, prog, fw_path)

And capture a new trace:

In [ ]:
trace2 = cw.capture_trace(scope, target, text)

In [ ]:
hv.Curve(trace2.wave).opts(width=600, height=600)

It should now be very obvious where our loop is happening. Verify that the number of similar looking power spikes matches how many times the loop is run. Now let's take a look at how different operations affect power traces:

## Comparing Operations

To make things easy on us, let's select operations that we would expect to have very different power traces. One easy metric to base your decision on is how long the associated instructions take to execute (which is typically documented either in the datasheet, or in the core's documentation). For example, on XMEGA we'll compare adding (1 cycle) with multiplying (2 cycles).

### XMEGA

### STM32F3

Our reference for the STM32F3 (which has an Arm Cortex M4 CPU) will the [Cortex M4 Instruction Set Summary](http://infocenter.arm.com/help/index.jsp?topic=/com.arm.doc.ddi0439b/CHDDIGAC.html). As we can see, the multiply instruction takes 1 cycle to execute, while the divide instruction takes between 2 and 12. These instructions also have analogous C operations (* and /) so it will be easy to get the compiler to insert them.

This is a pretty drastic difference, so the two instructions should look very different. Change your loop so that it preforms multiplications each time through:

```C
volatile int b = 0xAFFA;
for (volatile int i = 0; i < 10; i++)
    b *= 11;
```

### Capture: Operation 1

Now rebuild and capture another trace:

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
cw.program_target(scope, prog, fw_path)

In [ ]:
trace3 = cw.capture_trace(scope, target, text)
hv.Curve(trace3.wave).opts(width=600, height=600)

### Capture: Operation 2

Replace your first operation with your second. Save the file, rebuild, program, and capture another trace.

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
cw.program_target(scope, prog, fw_path)

In [ ]:
trace4 = cw.capture_trace(scope, target, text)
hv.Curve(trace4.wave).opts(width=600, height=600)

Compare the traces. Is the one that you expected to be longer actually longer?

The longer operation is typically more complicated than the other operation, which means we would expect it to have higher power consumptions as well. Do you also see this behaviour in the traces you captured?

## Disconnecting from ChipWhisperer

Now that we're done with the tutorial, we'll need to disconnect from ChipWhisperer, so that we can connect to it in a different tutorial:

In [ ]:
scope.dis()
target.dis()

## Conclusion

In this tutorial you have learned how power analysis can tell you the operations being performed on a microcontroller. In future work we will move towards using this for breaking various forms of security on devices. In particular, Tutorial B3-1 will examine how we can use this information to exploit a password check.